In [ ]:
# !pip install -e /files/shared/ap/jupyter-notebook/pkgs/objFuncs --break-system-packages
# !pip install -e /files/shared/ap/jupyter-notebook/pkgs/pyBO --break-system-packages

In [ ]:
# !pip install -e /files/shared/ap/jupyter-notebook/pkgs/objFuncs
# !pip install -e /files/shared/ap/jupyter-notebook/pkgs/pyBO

In [ ]:
# !pip install -e ../../objFuncs
# !pip install -e ../../pyBO

In [3]:
import IPython
import os

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import datetime
from phantasy import caget

In [5]:
import objFuncs
from objFuncs import residuals
from objFuncs.util import plot_obj_history
from objFuncs import flame_utils_kilean as flame_utils

# check machineIO, source and beam 

In [ ]:
objFuncs._global_machineIO._test = False
objFuncs._global_machineIO._fetch_data_time_span = 2.05
objFuncs._global_machineIO._ensure_set_timewait_after_ramp : 0.25
objFuncs._global_machineIO.view()

In [8]:
SCS = caget("ACS_DIAG:DEST:ACTIVE_ION_SOURCE")
ion = caget("FE_ISRC"+str(SCS)+":BEAM:ELMT_BOOK")
Q = caget("FE_ISRC"+str(SCS)+":BEAM:Q_BOOK")
A = caget("FE_ISRC"+str(SCS)+":BEAM:A_BOOK")
# AQ = caget("FE_ISRC2:BEAM:MOVRQ_BOOK")
AQ = A/Q
ion = str(A)+ion+str(Q)
print('SCS'+str(SCS), ion, 'A/Q=',AQ)

SCS1 16O8 A/Q= 2.0


# preprare objective (for residual construction)

In [9]:
%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [10]:
now0str = datetime.datetime.now().strftime('%Y%m%d_%H%M')
fname = now0str+'['+ion+']'+nb_name.strip('.ipynb')
fname

'20231215_1002[16O8][LSQ][FS2][before1stDipole]QuadCentering'

In [ ]:
decision_CSETs=[
    'FS2_BTS:PSC2_D3930:I_CSET',
    'FS2_BTS:PSC2_D3945:I_CSET',
]
decision_min = [-5*AQ]
decision_max = [ 5*AQ]

print(f'decision_min: {decision_min}')
print(f'decision_max: {decision_max}')

In [ ]:
objective_goal = { 
    'FS2_BTS:BPM_D4006:XPOS_RD' : 0.0,
    'FS2_BMS:BPM_D4142:XPOS_RD' : 0.0,
}
objective_norm = {pv:1.0 for pv in objective_goal.keys()}
objective_weight = {pv:1.0 for pv in objective_goal.keys()}
objective_RDs = list(objective_goal.keys())

In [ ]:
conditional_SETs = {
#     'FS2_BTS:PSQ_D3934:I_CSET':[80,100, 80],
#     'FS2_BTS:PSQ_D3940:I_CSET':[80, 80,100],
    'FS2_BTS:PSQ_D3950:I_CSET':[80, 80,100],
    'FS2_BTS:PSQ_D3955:I_CSET':[80, 80,100],
}

In [ ]:
res = residuals.residualObjMultiConditionalVar(  
    decision_CSETs = decision_CSETs,
    decision_min = decision_min,
    decision_max =  decision_max,
    objective_goal = objective_goal,
    objective_norm = objective_norm,
    objective_weight = objective_weight,
    conditional_SETs = conditional_SETs,
    var_obj_weight_fraction = 0.9,
    )

In [ ]:
plot_decision = plot_obj_history(
    res.history['decision_CSETs'],
    title='decision_CSETs',
)
plot_conditions_RDs = plot_multi_obj_history(
    histories = [res.history['condition'+str(i)]['objective_RDs'] for i in range(n_condition)],
    history_labels = ['C'+str(i) for i in range(n_condition)],
    title='multi-condition_obj_RDs',
)
plot_objectives_var = plot_obj_history(
    res.history['objectives_var'],
    title='objectives_var',
)
callbacks = [plot_decision,plot_conditions_RDs,plot_objectives_var]
# callbacks = []

In [ ]:
result = res.lsq_linear(jac_use3points=False,callbacks=callbacks)
for f in callbacks:
    f.close()

In [ ]:
for name, val in zip(res.history['jacobian_var']['names'], res.history['jacobian_var']['values'][0]):
    print( name, val )

In [ ]:
obj.write_log(fname=os.path.join('./log',fname))